In [17]:
import pandas as pd
import numpy as np

In [18]:
df=pd.read_csv("train.csv",engine="python",error_bad_lines=False)   ##Read the dataset and handle the error

<ipython-input-18-ed21837c92d8>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("train.csv",engine="python",error_bad_lines=False)   ##Read the dataset and handle the error
Skipping line 9676: unexpected end of data


In [19]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [20]:
df.shape

(9674, 5)

In [21]:
##check the missing values and handle it
df.isnull().sum()


id          0
title     264
author    922
text       21
label       0
dtype: int64

In [22]:
df=df.dropna()

In [23]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
df.shape

(8488, 5)

In [25]:
##independent feature
X=df.drop("label",axis=1)

In [26]:
##Dependent feature
y=df["label"]

In [27]:
X.shape

(8488, 4)

In [29]:
y.shape

(8488,)

In [30]:
##check whether the dataset is balanced or not
y.value_counts()                ##it is the balanced daatset

0    4833
1    3655
Name: label, dtype: int64

In [31]:
import tensorflow as tf
tf.__version__

'2.13.0'

In [32]:
from tensorflow.keras.layers import Embedding   ##for the creating the embedding layer
from tensorflow.keras.preprocessing.sequence import pad_sequences       ##for creating the padding sequence
from tensorflow.keras.models import Sequential    ##create the sequential model
from tensorflow.keras.preprocessing.text import one_hot        ##implementing the one_hot_representation
from tensorflow.keras.layers import LSTM               ##import the LSTM mode
from tensorflow.keras.layers import Dense              ##import the Dense for the output layer
from tensorflow.keras.layers import Bidirectional      ##import the Bidirectional to implement the Bidirectional LSTM model

In [33]:
## vocabulary size
voc_size=5000

In [34]:
messages=X.copy()

In [35]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [36]:
import nltk
import re
from nltk.corpus import stopwords

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
messages.reset_index(inplace=True)

In [39]:
##Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677

In [40]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [41]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]     ##it gives the index of each words present in the dataset
onehot_repr

[[4694, 3511, 2968, 2594, 3103, 2016, 3116, 3981, 3872, 1885],
 [231, 4818, 4625, 2522, 1502, 3978, 4868],
 [4185, 1691, 1461, 1994],
 [1374, 1147, 586, 2670, 4739, 4484],
 [3876, 1502, 4993, 3113, 4235, 4803, 1502, 3655, 2139, 4585],
 [4057,
  4826,
  937,
  3948,
  2282,
  1787,
  3944,
  2012,
  3882,
  1368,
  2778,
  2262,
  2027,
  4044,
  4868],
 [1285, 98, 4396, 437, 187, 3666, 1853, 1284, 3140, 2707, 442],
 [2908, 411, 4532, 485, 2188, 1079, 1787, 3505, 3140, 2707, 442],
 [3000, 2836, 470, 4281, 1485, 301, 720, 1942, 1787, 1173],
 [2618, 2129, 2039, 4437, 3963, 2481, 3385, 3404],
 [784, 2292, 2970, 3792, 2208, 4520, 1597, 351, 4422, 1729, 2125],
 [2670, 1800, 3103, 301, 1787, 2188],
 [4478, 2293, 2898, 1661, 4080, 3636, 1176, 2933, 3632],
 [3410, 1118, 4020, 4047, 2917, 2147, 3037, 3140, 2707, 442],
 [4648, 4673, 1494, 2914, 4141, 3140, 2707, 442],
 [1693, 3515, 740, 232, 1575, 44, 2417, 1570, 192, 3755],
 [2791, 1079, 4818],
 [539, 1344, 287, 1115, 1787, 4525, 2771, 4868],
 [

In [42]:
##Now, to convert this index into the vector we have to create the embedding layer
###Embedding Representation

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 3981, 3872, 1885],
       [   0,    0,    0, ..., 1502, 3978, 4868],
       [   0,    0,    0, ..., 1691, 1461, 1994],
       ...,
       [   0,    0,    0, ..., 2452, 1787, 4396],
       [   0,    0,    0, ..., 3140, 2707,  442],
       [   0,    0,    0, ...,  647, 3901, 2015]], dtype=int32)

In [43]:
##Creating the models
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [44]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [45]:
## train test split with test size is 0.33

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [47]:
## Model Training with epochs is 25 and batch size as 64
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=40,batch_size=64)

Epoch 1/40
89/89 [==============================] - 3s 38ms/step - loss: 3.0931e-05 - accuracy: 1.0000 - val_loss: 0.8685 - val_accuracy: 0.8969
Epoch 2/40
89/89 [==============================] - 3s 37ms/step - loss: 2.6944e-05 - accuracy: 1.0000 - val_loss: 0.8796 - val_accuracy: 0.8969
Epoch 3/40
89/89 [==============================] - 4s 46ms/step - loss: 2.3845e-05 - accuracy: 1.0000 - val_loss: 0.8892 - val_accuracy: 0.8969
Epoch 4/40
89/89 [==============================] - 3s 36ms/step - loss: 2.1353e-05 - accuracy: 1.0000 - val_loss: 0.8984 - val_accuracy: 0.8969
Epoch 5/40
89/89 [==============================] - 3s 37ms/step - loss: 1.9281e-05 - accuracy: 1.0000 - val_loss: 0.9068 - val_accuracy: 0.8969
Epoch 6/40
89/89 [==============================] - 5s 51ms/step - loss: 1.7525e-05 - accuracy: 1.0000 - val_loss: 0.9146 - val_accuracy: 0.8972
Epoch 7/40
89/89 [==============================] - 3s 37ms/step - loss: 1.6016e-05 - accuracy: 1.0000 - val_loss: 0.9223 - val_ac

In [48]:
y_pred=model.predict(X_test)  ##predict the y_pred value

88/88 [==============================] - 1s 5ms/step


In [49]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)    ##put that value as 1 whose pred_value>=0.5 and put that value as 0 ehose pred_value<0.5

In [50]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1474,  144],
       [ 145, 1039]])

In [52]:
print(accuracy_score(y_test,y_pred))    ##Find out the accuracy of the model

0.8968593861527481


In [ ]:
## Find out the accuracy score of the LSTM model i.e 89.69%